In [17]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
import pickle
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Load already augmented data from saved pickle files
X_train_aug = pickle.load(open("../../pickles/X_train_aug.pkl", "rb"))
y_train_aug = pickle.load(open("../../pickles/y_train_aug.pkl", "rb"))
X_test = pickle.load(open("../../pickles/X_test.pkl", "rb"))
y_test = pickle.load(open("../../pickles/y_test.pkl", "rb"))

In [ ]:
# Confirm the shapes of the data
display(X_train_aug.shape, y_train_aug.shape, X_test.shape, y_test.shape)

In [4]:
# further split the training data into training and validation
X_train, X_val, y_train, y_val = train_test_split(X_train_aug, y_train_aug, test_size=0.2, random_state=42)

In [ ]:
# Create the model using Convolutional Neural Network
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu'),   # 32 filters, 3x3 kernel
    layers.MaxPooling2D((2, 2)),                    # 2x2 pool size
    layers.Conv2D(64, (3, 3), activation='relu'),   # 64 filters, 3x3 kernel
    layers.MaxPooling2D((2, 2)),                    # 2x2 pool size
    layers.Conv2D(64, (3, 3), activation='relu'),   # 64 filters, 3x3 kernel       
    layers.Flatten(),                               # Flatten the 3D output to 1D
    layers.Dense(64, activation='relu'),            # 64 neurons
    layers.Dense(6, activation='softmax')           # 6 classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 10
history = model.fit(
    X_train, y_train, 
    batch_size=batch_size, 
    epochs=epochs, 
    validation_data=(X_val, y_val))

In [ ]:
# Evaluate the model
model.evaluate(X_test, y_test)

In [15]:
def plot_training_history(history):
    # Extracting training history
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1)

    # Create subplots for accuracy and loss
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))

    # Plot accuracy
    axs[0].plot(epochs, acc, 'bo-', label='Training Accuracy')
    axs[0].plot(epochs, val_acc, 'r*-', label='Validation Accuracy')
    axs[0].set_title('Training & Validation Accuracy')
    axs[0].set_xlabel('Epochs')
    axs[0].set_ylabel('Accuracy')
    axs[0].legend()

    # Plot loss
    axs[1].plot(epochs, loss, 'bo-', label='Training Loss')
    axs[1].plot(epochs, val_loss, 'r*-', label='Validation Loss')
    axs[1].set_title('Training & Validation Loss')
    axs[1].set_xlabel('Epochs')
    axs[1].set_ylabel('Loss')
    axs[1].legend()

    plt.tight_layout()
    plt.show()

In [ ]:
plot_training_history(history)

In [ ]:
df = pd.DataFrame(history.history)

# Plot validation accuracy and validation loss on the same line chart
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(df) + 1) , df['val_accuracy'], marker='o', linestyle='-', color='blue', label='Validation Accuracy')
plt.plot(range(1, len(df) + 1), df['val_loss'], marker='s', linestyle='--', color='red', label='Validation Loss')

plt.xlabel('Epochs')
plt.ylabel('Value')
plt.title('Validation Accuracy & Loss over Epochs')
plt.legend()
plt.grid(True)
plt.show()

In [7]:
model.save("../../models/model_softmax_ohe_10e.keras")

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, to_file='model_softmax_ohe_10e.png',
                          show_shapes=True,
                          show_dtype=False,
                          show_layer_names=True,
                          rankdir="TB",
                          expand_nested=False,
                          dpi=96)

In [ ]:
import visualkeras 
visualkeras.layered_view(model)
